In [1]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import pyarrow as pa
import csv
import subprocess

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.1.0 pyspark-shell'

#spark = SparkSession.builder.appName('SparkCassandraApp').master("local[*]").getOrCreate()
import warnings
warnings.filterwarnings('ignore')

In [2]:
KeyspaceName = 'ukrainetweets'
TableName = 'RawTweet'
session.set_keyspace(KeyspaceName)
#session.execute(f'DROP TABLE {TableName}')

In [3]:
import os
import csv
import gzip
from cassandra.cluster import Cluster
import pandas as pd


ReplicationOptions = {'class': 'SimpleStrategy', 'replication_factor': 1}

query = f"CREATE KEYSPACE IF NOT EXISTS {KeyspaceName} WITH REPLICATION = {ReplicationOptions}"
session.execute(query)

TableName = 'RawTweet'
ColumnNames = ["num", "userid","username","acctdesc","location","following","followers","totaltweets","usercreatedts","tweetid","tweetcreatedts","retweetcount","text","hashtags","language","coordinates","favorite_count","is_retweet","original_tweet_id","original_tweet_userid","original_tweet_username","in_reply_to_status_id","in_reply_to_user_id","in_reply_to_screen_name","is_quote_status","quoted_status_id","quoted_status_userid","quoted_status_username","extractedts"]

create_table = f"CREATE TABLE IF NOT EXISTS {KeyspaceName}.{TableName} (num TEXT, userid TEXT, username TEXT, acctdesc TEXT, location TEXT, following TEXT, followers TEXT, totaltweets TEXT, usercreatedts TIMESTAMP, tweetid TEXT PRIMARY KEY, tweetcreatedts TIMESTAMP, retweetcount TEXT, text TEXT, hashtags TEXT, language TEXT, coordinates TEXT, favorite_count TEXT, is_retweet TEXT, original_tweet_id TEXT, original_tweet_userid TEXT, original_tweet_username TEXT, in_reply_to_status_id TEXT, in_reply_to_user_id TEXT, in_reply_to_screen_name TEXT, is_quote_status TEXT, quoted_status_id TEXT, quoted_status_userid TEXT, quoted_status_username TEXT, extractedts TEXT)"
session.execute(create_table)






In [ ]:
direct = '/home/hduser/Desktop/CA2/CA2'
I = 0
tweets = []
keywords = ['refugee', 'refugees', 'asylum', 'asylum seeker', 'asylum seeking', 'migration', 'refugee crisis']
for file in os.listdir(direct):
    I+=1
    print(I)
    if file.endswith('.csv.gzip'):
        path = os.path.join(direct, file)
        
        with gzip.open(path, 'rt', encoding = 'utf-8') as file:
            reader = csv.reader(file)
            header = next(reader)
            text_index = header.index('text')
            for row in reader:
                if any(keyword in row[text_index].lower() for keyword in keywords):
                    tweets.append(row)
        
df = pd.DataFrame(tweets)
df.head()

df.to_csv('Filtered_Tweets.csv', index = False)

#The cassandra drivers seem to not support the copy command so this was done in the terminal using the following command
'''
COPY RawTweet (num, userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,is_retweet,original_tweet_id,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts)FROM '/home/hduser/Desktop/CA2/Filtered_Tweets.csv'

'''

Number of reccords in the cassandra database

In [4]:
count = session.execute('SELECT COUNT(*) FROM RawTweet')
print(count[0])

Row(count=504771)


In [5]:
spark

In [6]:
from pyspark.sql.functions import col, to_timestamp
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#import pyspark_cassandra
#pyspark --packages com.datastax.spark:spark-cassandra-connector_2.12:3.1.0

#spark.conf.set(s"spark.sql.catalog.cass100", "com.datastax.spark.connector.datasource.CassandraCatalog")
#spark.conf.set(s"spark.sql.catalog.cass100.spark.cassandra.connection.host", "127.0.0.100")

In [7]:
refugee = spark.read.format("org.apache.spark.sql.cassandra").options(table='rawtweet', keyspace=KeyspaceName).load().select("tweetid", "tweetcreatedts", "text", "language", "location")
#df = refugee.toPandas()

#from dateutil.parser import parse

#df['tweetcreatedts'] = df['tweetcreatedts'].apply(lambda x: parse(x))
#df.info()

In [8]:
refugee.show(20)

+-------------------+-------------------+--------------------+--------+--------------------+
|            tweetid|     tweetcreatedts|                text|language|            location|
+-------------------+-------------------+--------------------+--------+--------------------+
|1518782028606550017|2022-04-26 03:40:27|Before the #Ukrai...|      en|                JoJa|
|1526454581252739073|2022-05-17 07:48:26|In #Tigray the #C...|      en|                null|
|1544061203437207554|2022-07-04 21:50:52|Welcome to the wo...|      en|Mashonaland Centr...|
|1637234348905693185|2023-03-18 23:27:41|I am alarmed &amp...|      en|Leicester & Manch...|
|1504581442642354183|2022-03-17 22:12:24|Stories told by #...|      en|    Toronto, Ontario|
|1502098341290782726|2022-03-11 01:45:26|#Ukraine crisis c...|      en|                null|
|1643934480820850688|2023-04-06 12:11:37|Listen to this go...|      en|       Edinburgh, UK|
|1576820096185425920|2022-10-03 07:23:01|In Vorbereitung a...|      de

2023-05-23 09:48:59,740 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-2466941506363674373, "partition key token"=-2025346008244648701]' generated server side warning(s): Read 1000 live rows and 8934 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > -2466941506363674373 AND token(tweetid) <= -2025346008244648701 LIMIT 1000; token -2431622458698931965 (see tombstone_warn_threshold)
2023-05-23 09:48:59,755 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-2466941506363674373, "partition key token"=-2025346008244648701]' generated server side warn

In [9]:
#remove non english comments
from pyspark.sql.functions import col
refugee = refugee.filter(col('language') == 'en')
refugee.show(20)


+-------------------+-------------------+--------------------+--------+--------------------+
|            tweetid|     tweetcreatedts|                text|language|            location|
+-------------------+-------------------+--------------------+--------+--------------------+
|1536690832757096448|2022-06-14 13:43:39|🇬🇧|#Migration: ...|      en|                null|
|1507045957669687304|2022-03-24 17:25:30|🙏🏽 #OrlandoBloo...|      en|              Europe|
|1516221156780511237|2022-04-19 02:04:28|.@JoyAnnReid lead...|      en|                null|
|1505778534966517760|2022-03-21 05:29:13|More than 360.000...|      en|                null|
|1501902545836716034|2022-03-10 12:47:25|Meet the Kerala c...|      en|           New Delhi|
|1497571486286155776|2022-02-26 13:57:20|@simon_harper @er...|      en|                  UK|
|1589290428704391168|2022-11-06 16:15:40|⚡️Ukrainian and I...|      en|                null|
|1500986201968234496|2022-03-08 00:06:12|Request: Helping ...|      en|   

2023-05-23 09:49:04,526 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s): Read 1000 live rows and 8991 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > 7447475080679012599 AND token(tweetid) <= 8024891708407858539 LIMIT 1000; token 7483727941932587962 (see tombstone_warn_threshold)
2023-05-23 09:49:04,536 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=7447475080679012599, "partition key token"=8024891708407858539]' generated server side warning(s):

In [10]:
from pyspark.sql.functions import udf, lower, col, regexp_replace, concat_ws
from pyspark.sql.types import StringType, FloatType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # for tokenization
nltk.download('wordnet')  # for lemmatization
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hduser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hduser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
#remove retweets
refugee = refugee.filter(~col('text').rlike('RT @%'))
#make lower case
refugee = refugee.withColumn('text', lower(col('text')))
#removing twitter name
refugee = refugee.withColumn('text', regexp_replace(col('text'), '@[^\s]+', ''))
#removing hashtag
refugee = refugee.withColumn('text', regexp_replace(col('text'), '\\\B#\\S+', ''))
#removing special characters nums and punctuation
refugee = refugee.withColumn('text', regexp_replace(col('text'), '[^A-Za-z0-9 ]+', ''))
#remove single characters
refugee = refugee.withColumn('text', regexp_replace(col('text'), '\\s+[a-zA-Z]\\s+', ''))

#Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    words = nltk.word_tokenize(text)
    lemmatized = ' '.join([lemmatizer.lemmatize(word) for word in words])
    return lemmatized
#make into udf
lemmatize_udf = udf(lemmatize, StringType())
refugee = refugee.withColumn('text', lemmatize_udf('text'))
#Tokenize words, this is done as the stop words remover needs an array of words
tokenizer = Tokenizer(inputCol="text", outputCol="words")
refugee = tokenizer.transform(refugee)
# Removing stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
refugee = remover.transform(refugee)

#recombine the words into text
refugee = refugee.withColumn('text', concat_ws(' ', col("filtered_words")))


In [12]:
refugee.show(20)

2023-05-23 09:49:15,990 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-4708906719179708351, "partition key token"=-3857806206306463357]' generated server side warning(s): Read 1000 live rows and 8829 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > -4708906719179708351 AND token(tweetid) <= -3857806206306463357 LIMIT 1000; token -4672053828842050133 (see tombstone_warn_threshold)
2023-05-23 09:49:16,002 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-4708906719179708351, "partition key token"=-3857806206306463357]' generated server side warn

+-------------------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+
|            tweetid|     tweetcreatedts|                text|language|            location|               words|      filtered_words|
+-------------------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+
|1505645038986010624|2022-03-20 20:38:45|amp mila kunis am...|      en|                null|[amp, mila, kunis...|[amp, mila, kunis...|
|1513000525490712579|2022-04-10 04:46:50|824 million peopl...|      en|Home of KTel Reco...|[824, million, pe...|[824, million, pe...|
|1505268701515427843|2022-03-19 19:43:19|know shabbat rabb...|      en|        D.C. Suburbs|[i, know, it, is,...|[know, shabbat, r...|
|1527951620394336256|2022-05-21 10:57:08|todaymet beautifu...|      en|                null|[todaymet, the, m...|[todaymet, beauti...|
|1504370285029601281|2022-03-17 08:13:20|four incredibl

2023-05-23 09:49:16,941 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-4708906719179708351, "partition key token"=-3857806206306463357]' generated server side warning(s): Read 1000 live rows and 8717 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > token(1577232598052769792) AND token(tweetid) <= -3857806206306463357 LIMIT 1000; token -4116068240207905940 (see tombstone_warn_threshold)
2023-05-23 09:49:16,975 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-4708906719179708351, "partition key token"=-3857806206306463357]' generated server sid

In [27]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
def sentiment(text):
    sentiment_analy = SentimentIntensityAnalyzer()
    vader = sentiment_analy.polarity_scores(text)
    return float(vader['compound'])
    

In [15]:
udfSentiment = udf(sentiment, FloatType())
refugee = refugee.withColumn('sentiment polarity', udfSentiment('text'))
refugee.persist()
refugee.show(20)


2023-05-23 09:49:37,001 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-3006705693433218363, "partition key token"=-2466941506363674373]' generated server side warning(s): Read 1000 live rows and 8713 tombstone cells for query SELECT language, location, text, tweetcreatedts FROM ukrainetweets.rawtweet WHERE token(tweetid) > -3006705693433218363 AND token(tweetid) <= -2466941506363674373 LIMIT 1000; token -2971044106971670173 (see tombstone_warn_threshold)
2023-05-23 09:49:37,013 WARN cql.CqlRequestHandler: Query '[2 values] SELECT "tweetid", "language", "location", "text", "tweetcreatedts" FROM ukrainetweets.rawtweet WHERE token("tweetid") > ? AND token("tweetid") <= ?   ALLOW FILTERING ["partition key token"=-3006705693433218363, "partition key token"=-2466941506363674373]' generated server side warn

KeyboardInterrupt: 

In [16]:
#maker functioin to catagorize sentiment
def sentiment_cat(sentiment):
    if sentiment <= -0.6:
        return "Negative"
    elif -0.6 < sentiment <= -0.2:
        return "Ngative-ish"
    elif -0.2 < sentiment <= 0.2:
        return "Neutral"
    elif 0.2 < sentiment <= 0.6:
        return "Positive-ish"
    else:
        return "Positive"
sentiment_cat_udf = udf(sentiment_cat, StringType())
refugee = refugee.withColumn('Sentiment_Category', sentiment_cat_udf('sentiment polarity'))
refugee.persist()
refugee.show()


+-------------------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+------------------+
|            tweetid|     tweetcreatedts|                text|language|            location|               words|      filtered_words|sentiment polarity|Sentiment_Category|
+-------------------+-------------------+--------------------+--------+--------------------+--------------------+--------------------+------------------+------------------+
|1518315286520209408|2022-04-24 20:45:47|2 month russian w...|      en|England, United K...|[it, 2, month, of...|[2, month, russia...|           -0.7906|          Negative|
|1508452633346215936|2022-03-28 15:35:08|around 38 million...|      en|                null|[around, 38, mill...|[around, 38, mill...|           -0.5994|       Ngative-ish|
|1525249861242830848|2022-05-14 00:01:19|hospital tigray d...|      en|                null|[hospital, in, ti...|[hospital, tigray...| 

In [24]:
from pyspark.sql.functions import avg, date_format

timeseries = refugee.withColumn('date', date_format('tweetcreatedts', 'yyyy-MM-dd'))
timeseries = timeseries.withColumn('hour', date_format('tweetcreatedts', 'HH'))
timeseries = timeseries.select('date', 'hour', 'sentiment polarity')

In [25]:
timeseries = timeseries.groupBy('date', 'hour').agg(avg('sentiment polarity').alias('hourly_polarity'))

In [ ]:
timeseries = timeseries.orderBy(col('date'), col('hour'))
timeseries.persist()
timeseries.show(20)

2023-05-23 09:57:53,760 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 09:57:53,836 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 09:58:38,495 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 09:58:38,495 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.nett

2023-05-23 10:00:37,330 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:00:37,330 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.nett

2023-05-23 10:02:29,357 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:02:37,348 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 10:04:29,361 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:04:37,355 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

...

2023-05-23 10:40:45,762 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:41:28,582 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 10:42:40,974 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:43:12,999 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 10:44:38,578 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:45:07,608 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 10:46:38,582 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:47:07,644 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

2023-05-23 10:48:38,586 WARN control.ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=acff7f38-e521-4f7c-9d41-f923710d4c18, hashCode=486e3984), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-Cassandra-Connector-local-1684831698830}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
2023-05-23 10:49:07,651 WARN pool.ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=5c2c140c-d6d2-4773-8e1c-9c9ea8748d3e, APPLICATION_NAME=Spark-C

In [ ]:
null_rows = refugee.filter(col('tweetcreatedts').isNull())
null_rows.count()
